# Kronig-Penney Model
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/model/kronig-penney_model/)

The Kronig-Penney model is a 1D system that demonstrates band gaps, which relate to the allowed energies for electrons in a material. In this tutorial we calculate the bandstructure for Kronig-Penney Model. The Kronig-Penney Model has a periodic potential of

$$
V(x) =
\begin{cases}
      V_0 & -b < x < 0 \cr
      0 & 0 < x < a
\end{cases}
$$
Where b is the width of each barrier, and a is the spacing between them.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from postopus import Run

In [ ]:
!mkdir 5-Kronig-Penney-Model

In [ ]:
cd 5-Kronig-Penney-Model 

## Input
The following input file will be used for the ground state calculation:

In [ ]:
%%writefile inp

stdout = 'stdout.txt'
stderr = 'stderr.txt'

CalculationMode = gs
ExtraStates = 4
PeriodicDimensions = 1
Dimensions = 1
TheoryLevel = independent_particles

a = 5
b = 1
V = 3

Lsize = (a + b)/2

%LatticeParameters
  a + b
%

Spacing = .0075

%Species
 "A" | species_user_defined | potential_formula | "(x>-b)*V*(x<0)" | valence | 1
%

%Coordinates
 "A" | 0 |
%

%KPointsGrid
 11 |
%

%KPointsPath
  11 |
 0.0 |
 0.5 |
%

ConvEigenError = true

%Output
 potential
 wfs
%
OutputFormat = axis_x
OutputWfsNumber = "1,2"

In [ ]:
!octopus

In [ ]:
run = Run(".")

In [ ]:
wf = run.default.scf.wf().isel(step=-1)
wf.sel(st=1, k=1).plot(label="1st state", color="green")
wf.sel(st=2, k=1).plot(label="2nd state", color="blue")
x = np.linspace(-3, 3, 1000)
plt.plot(
    x,
    np.where((x >= -1) & (x <= 0), 3, 0),
    label="potential",
    color="purple",
    linewidth=1,
)
plt.title("The first two wavefunctions plotted alongside the potential. \n")
plt.ylabel("")
plt.ylim(-1, 4)
plt.xlim(-3, 3)
plt.grid(True)
plt.legend()
plt.show()

## Bandstructure

To calculate the bandstructure simply change the [CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) to unocc.

In [ ]:
%%writefile inp

stdout = 'stdout.txt'
stderr = 'stderr.txt'

CalculationMode = unocc
ExtraStates = 4
PeriodicDimensions = 1
Dimensions = 1
TheoryLevel = independent_particles

a = 5
b = 1
V = 3

Lsize = (a + b)/2

%LatticeParameters
 a + b
%

Spacing = .0075

%Species
 "A" | species_user_defined | potential_formula | "(x>-b)*V*(x<0)" | valence | 1
%

%Coordinates
 "A" | 0 |
%

%KPointsGrid
 11 |
%

%KPointsPath
 11 |
 0.0 |
 0.5 |
%

ConvEigenError = true

In [ ]:
!octopus

To plot the bandstructure, we will use postopus.

In [ ]:
run.default.scf.bandstructure().iloc[:, 3:6].plot(
    color="purple", linewidth=1, legend=False
)
plt.xlim(0, 1)
plt.ylim(1, 4)
plt.xlabel(r"$k \left(\frac{a+b}{\pi}\right)$")
plt.ylabel("Energy [Hartree]")
plt.title("The band structure for Kronig-Penney Model.")
plt.show()

[Back to main tutorial series](../Readme.ipynb)

## References

1. Sidebottom DL. Fundamentals of condensed matter and crystalline physics: an introduction for students of physics and materials science. New York: Cambridge University Press; 2012.